In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.callbacks import EarlyStopping 

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, TimeDistributed, Conv1D, GlobalMaxPool1D, MaxPooling1D, SimpleRNN, LSTM, GRU

In [3]:
import json
import random
from collections import Counter, namedtuple
import razdel
import pymorphy2
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
import lexrank
from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
import nltk; nltk.download('punkt');
from summa.summarizer import summarize
import copy

from itertools import combinations
import networkx as nx
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\snetkova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r", encoding='utf-8') as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [5]:
train_records = read_gazeta_records("D:/GeekBrains/nlp/les12/data/gazeta_train.jsonl")
val_records = read_gazeta_records("D:/GeekBrains/nlp/les12/data/gazeta_val.jsonl")
test_records = read_gazeta_records("D:/GeekBrains/nlp/les12/data/gazeta_test.jsonl")

In [6]:
train_records[0]

{'url': 'https://www.gazeta.ru/financial/2011/11/30/3852658.shtml',
 'text': '«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожид

In [7]:
Stats = namedtuple("Stats", "vocabulary, lemma_vocabulary, words_counts, unique_words_counts")

def collect_stats(records, lower=True, text_max_words=3000, summary_max_words=100, nrows=1000):
    morph = pymorphy2.MorphAnalyzer()
    
    text_stats = Stats(Counter(),  Counter(), list(), list())
    summary_stats = Stats(Counter(),  Counter(), list(), list())

    def update_record_field_stats(field, stats, max_words):
        words = [word.text for word in razdel.tokenize(field)][:max_words]
        lemmas = [morph.parse(word)[0].normal_form for word in words]
        stats.vocabulary.update(words)
        stats.lemma_vocabulary.update(lemmas)
        stats.words_counts.append(len(words))
        stats.unique_words_counts.append(len(set(words)))

    for i, record in enumerate(records):
        if i >= nrows:
            break
        text = record["text"]
        text = text if not lower else text.lower()
        update_record_field_stats(text, text_stats, text_max_words)

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        summary_words = [word.text for word in razdel.tokenize(summary)]
        update_record_field_stats(summary, summary_stats, summary_max_words)
    return text_stats, summary_stats

In [8]:
train_text_stats, train_summary_stats = collect_stats(train_records)
print("Train texts vocabulary size: ", len(train_text_stats.vocabulary))
print("Train texts lemma vocabulary size: ", len(train_text_stats.lemma_vocabulary))
print("Train summaries vocabulary size: ", len(train_summary_stats.vocabulary))
print("Train summaries lemma vocabulary size: ", len(train_summary_stats.lemma_vocabulary))
print("Train common lemmas summary vs text: ", len(set(train_text_stats.lemma_vocabulary.keys()) & set(train_summary_stats.lemma_vocabulary.keys())))

Train texts vocabulary size:  82978
Train texts lemma vocabulary size:  37851
Train summaries vocabulary size:  15441
Train summaries lemma vocabulary size:  9030
Train common lemmas summary vs text:  8652


In [9]:
res = {}

### Lead-3

первые 3 предложения текста в качестве summary.

In [10]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        bl = corpus_bleu([[r] for r in references], predictions)
        print("BLEU: ", bl)
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)
    return bl, scores

In [11]:
def calc_lead_n_score(records, n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n]) # берем первые n предложений из текста
        predictions.append(prediction)

    return calc_scores(references, predictions)

In [12]:
calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'f': 0.23804097238957525, 'p': 0.22208274285774904, 'r': 0.37762764047433917}, 'rouge-2': {'f': 0.10027796832321115, 'p': 0.09647636782929753, 'r': 0.15833772153385062}, 'rouge-l': {'f': 0.1835646488408507, 'p': 0.2022959168891477, 'r': 0.34937017731940756}}


(0.19177311186434495,
 {'rouge-1': {'f': 0.23804097238957525,
   'p': 0.22208274285774904,
   'r': 0.37762764047433917},
  'rouge-2': {'f': 0.10027796832321115,
   'p': 0.09647636782929753,
   'r': 0.15833772153385062},
  'rouge-l': {'f': 0.1835646488408507,
   'p': 0.2022959168891477,
   'r': 0.34937017731940756}})

In [13]:
calc_lead_n_score(test_records, n=3)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области. «здесь только ты и бог. проект-откровение. узнай себя и мир монастырской жизни. одна из красивейших обителей россии на озере селигер. проживи месяц в ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект. в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. желающим предлагается 

(0.09837051890626336,
 {'rouge-1': {'f': 0.15856848565249815,
   'p': 0.09604033126642082,
   'r': 0.5566639977563598},
  'rouge-2': {'f': 0.05580674432615294,
   'p': 0.03297573363580163,
   'r': 0.23085498752032227},
  'rouge-l': {'f': 0.09403812710018163,
   'p': 0.09058743542400316,
   'r': 0.5266851606875759}})

Если брать только первое предложение в качестве саммари, метрики получились повыше

In [14]:
rs = calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'f': 0.23804097238957525, 'p': 0.22208274285774904, 'r': 0.37762764047433917}, 'rouge-2': {'f': 0.10027796832321115, 'p': 0.09647636782929753, 'r': 0.15833772153385062}, 'rouge-l': {'f': 0.1835646488408507, 'p': 0.2022959168891477, 'r': 0.34937017731940756}}


In [15]:
res['lead-1'] = rs[0], rs[1]['rouge-1']['f']
res

{'lead-1': (0.19177311186434495, 0.23804097238957525)}

### Textrank

1. Сплитим текст по предложениям
2. Считаем "похожесть" предложений между собой
3. Строим граф предложений с взвешенными ребрами
4. С помощью алгоритм PageRank получаем наиболее важные предложения, на основе которых делаем summary.

In [16]:
# можно брать другие меры близости предложений: по FastText, ELMo или BERT эмбеддингам
def unique_words_similarity(words1, words2):
    '''
    Функция подсчёта близости предложений на основе пересечения слов
    ''' 
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2)) / (len(words1) + len(words2))

In [17]:
def gen_text_rank_summary(text, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    # Токенизируем предложения
    sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]

    # При необходимости лемматизируем слова
    if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)
    scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])
    
    num_sents = len(result)
    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    
    return predicted_summary, num_sents

In [18]:
def calc_text_rank_score(records, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, nrows=1000, morph=None):
    references = []
    predictions = []
    length_sents = [] 

    for i, record in tqdm(enumerate(records)):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        predicted_summary, num_sents = gen_text_rank_summary(text, calc_similarity, summary_part, lower, morph=morph)
        length_sents.append(num_sents)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)

    print(f'Mean number of sents in summary: {np.mean(length_sents)}')
    return calc_scores(references, predictions)

In [19]:
morph = pymorphy2.MorphAnalyzer()
calc_text_rank_score(test_records)

0it [00:00, ?it/s]

Mean number of sents in summary: 3.276
Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. съемки начнутся уже в ноябре», — указывается в анонсе. от «зенита» до «спартака», — сказал священник. в июне нынешнего года сообщалось, что рпц планирует сделать из сергиева посада «православный ватикан».
BLEU:  0.3508755059541153
ROUGE:  {'rouge-1': {'f': 0.15466400887322831, 'p': 0.16298929023222172, 'r': 0.15602281048791292}, 'rouge-2': {'f': 0.030615979709094604, 'p': 0.031077035455225686, 'r': 0.032019614611385826}, 'rouge-l': {'f': 0.13373567967669006, 'p': 0.14775573752428653, 'r': 0.1

(0.3508755059541153,
 {'rouge-1': {'f': 0.15466400887322831,
   'p': 0.16298929023222172,
   'r': 0.15602281048791292},
  'rouge-2': {'f': 0.030615979709094604,
   'p': 0.031077035455225686,
   'r': 0.032019614611385826},
  'rouge-l': {'f': 0.13373567967669006,
   'p': 0.14775573752428653,
   'r': 0.14127533878751564}})

In [20]:
calc_text_rank_score(test_records, summary_part=0.2)

0it [00:00, ?it/s]

Mean number of sents in summary: 7.051
Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: проживи месяц в ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект. в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. «остров, который может изменить твою судьбу. съемки начнутся уже в ноябре», — указывается в анонсе. все зависит от людей, от их чуткости. а потом выпили, закусили, потом баня, рыбалка, потом «зенит», «спартак». от «зенита» до «спартака», — сказал священник. протоиерей призвал мужчин пройтись по улице, посмотреть на храмы, и почитать книги. в июне н

(0.24144496741046587,
 {'rouge-1': {'f': 0.16130641116593086,
   'p': 0.1232700823980428,
   'r': 0.24794461077173466},
  'rouge-2': {'f': 0.036863160694118796,
   'p': 0.02697858383367456,
   'r': 0.062123399348864516},
  'rouge-l': {'f': 0.12450680045547442,
   'p': 0.11342956259750214,
   'r': 0.22823635791987962}})

In [21]:
calc_text_rank_score(test_records, summary_part=0.1, morph = morph)

0it [00:00, ?it/s]

Mean number of sents in summary: 3.276
Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. съемки начнутся уже в ноябре», — указывается в анонсе. от «зенита» до «спартака», — сказал священник. по словам патриарха, им станет тот, кто будет стоять «во главе всемирной паутины».
BLEU:  0.3554246463687909
ROUGE:  {'rouge-1': {'f': 0.166697536373269, 'p': 0.16817668927460613, 'r': 0.1757661335817694}, 'rouge-2': {'f': 0.0372119139899679, 'p': 0.03638412165520413, 'r': 0.04094605692338856}, 'rouge-l': {'f': 0.14371031932437214, 'p': 0.15208339348514685, 'r': 0.1587150127702827}}


(0.3554246463687909,
 {'rouge-1': {'f': 0.166697536373269,
   'p': 0.16817668927460613,
   'r': 0.1757661335817694},
  'rouge-2': {'f': 0.0372119139899679,
   'p': 0.03638412165520413,
   'r': 0.04094605692338856},
  'rouge-l': {'f': 0.14371031932437214,
   'p': 0.15208339348514685,
   'r': 0.1587150127702827}})

In [22]:
rs = calc_text_rank_score(test_records, summary_part=0.2, morph = morph)

0it [00:00, ?it/s]

Mean number of sents in summary: 7.051
Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. «остров, который может изменить твою судьбу. съемки начнутся уже в ноябре», — указывается в анонсе. трудно сказать, к чему это приведет, поживем — увидим. а потом выпили, закусили, потом баня, рыбалка, потом «зенит», «спартак». от «зенита» до «спартака», — сказал священник. в июне нынешнего года сообщалось, что рпц планирует сделать из сергиева посада «православный ватикан». «если человеку невоцерковленному, далекому от церкви, в двух словах сказать, для чего нужна церковь — церковь… нужна дл

In [23]:
res['textrank'] = rs[0], rs[1]['rouge-1']['f']
res

{'lead-1': (0.19177311186434495, 0.23804097238957525),
 'textrank': (0.23481140598760156, 0.1709577523245075)}

Лемматизация сильно не помогла, увеличение длины саммари просаживает bleu, но почти не увеличивает rouge. Зато при увеличении длины саммари по тексту хотя бы становится понятно, о чем идет речь в исходной статье

### Lexrank

In [24]:
def calc_method_score(records, predict_func, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if nrows is not None and i >= nrows:
            break
        summary = record["summary"]
        text = record["text"]
        prediction = predict_func(text, summary)
        references.append(summary)
        predictions.append(prediction)

    return calc_scores(references, predictions)

In [25]:
def predict_lex_rank(text, summary, lxr, summary_size=3, threshold=None):
    sentences = [s.text for s in razdel.sentenize(text)]
    prediction = lxr.get_summary(sentences, summary_size=summary_size, threshold=threshold)
    prediction = " ".join(prediction)
    return prediction

In [26]:
sentences = [[s.text for s in razdel.sentenize(r["text"])] for r in test_records]
lxr = LexRank(sentences, stopwords=STOPWORDS['ru'])
calc_method_score(test_records, lambda x, y: predict_lex_rank(x, y, lxr))

Count: 1000
Ref: Телеканал «Спас» запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер. Организаторы отметили, что это беспрецедентный подобный проект на телевидении. Участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: РПЦ же нередко критикует СМИ. Критиковали в РПЦ и популярность футбола – об этом заявлял также Смирнов. В июне нынешнего года сообщалось, что РПЦ планирует сделать из Сергиева Посада «православный Ватикан».
BLEU:  0.3585872220880093
ROUGE:  {'rouge-1': {'f': 0.1866598392888999, 'p': 0.18137864753781832, 'r': 0.2055701577098893}, 'rouge-2': {'f': 0.05775558829470266, 'p': 0.05407660852215697, 'r': 0.06670656927927147}, 'rouge-l': {'f': 0.16173754022433068, 'p': 0.16619633248764337, 'r': 0.18766952121068614}}


(0.3585872220880093,
 {'rouge-1': {'f': 0.1866598392888999,
   'p': 0.18137864753781832,
   'r': 0.2055701577098893},
  'rouge-2': {'f': 0.05775558829470266,
   'p': 0.05407660852215697,
   'r': 0.06670656927927147},
  'rouge-l': {'f': 0.16173754022433068,
   'p': 0.16619633248764337,
   'r': 0.18766952121068614}})

In [27]:
calc_method_score(test_records, lambda x, y: predict_lex_rank(x, y, lxr, summary_size=1))

Count: 1000
Ref: Телеканал «Спас» запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер. Организаторы отметили, что это беспрецедентный подобный проект на телевидении. Участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: РПЦ же нередко критикует СМИ.
BLEU:  0.11823760201681352
ROUGE:  {'rouge-1': {'f': 0.1403400341975231, 'p': 0.24059238660376261, 'r': 0.10579274844085526}, 'rouge-2': {'f': 0.04389614671480771, 'p': 0.07527062641466548, 'r': 0.03322595135919795}, 'rouge-l': {'f': 0.10139954458840535, 'p': 0.2162630636902965, 'r': 0.09393481346181773}}


(0.11823760201681352,
 {'rouge-1': {'f': 0.1403400341975231,
   'p': 0.24059238660376261,
   'r': 0.10579274844085526},
  'rouge-2': {'f': 0.04389614671480771,
   'p': 0.07527062641466548,
   'r': 0.03322595135919795},
  'rouge-l': {'f': 0.10139954458840535,
   'p': 0.2162630636902965,
   'r': 0.09393481346181773}})

Здесь уменьшение длины саммари, наоборот, ухудшает метрики. В тексте саммари про запуск реалити-шоу нет ни слова

In [28]:
rs = calc_method_score(test_records, lambda x, y: predict_lex_rank(x, y, lxr))
res['lextrank'] = rs[0], rs[1]['rouge-1']['f']
res

Count: 1000
Ref: Телеканал «Спас» запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер. Организаторы отметили, что это беспрецедентный подобный проект на телевидении. Участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: РПЦ же нередко критикует СМИ. Критиковали в РПЦ и популярность футбола – об этом заявлял также Смирнов. В июне нынешнего года сообщалось, что РПЦ планирует сделать из Сергиева Посада «православный Ватикан».
BLEU:  0.3585872220880093
ROUGE:  {'rouge-1': {'f': 0.1866598392888999, 'p': 0.18137864753781832, 'r': 0.2055701577098893}, 'rouge-2': {'f': 0.05775558829470266, 'p': 0.05407660852215697, 'r': 0.06670656927927147}, 'rouge-l': {'f': 0.16173754022433068, 'p': 0.16619633248764337, 'r': 0.18766952121068614}}


{'lead-1': (0.19177311186434495, 0.23804097238957525),
 'textrank': (0.23481140598760156, 0.1709577523245075),
 'lextrank': (0.3585872220880093, 0.1866598392888999)}

### LSA

In [29]:
def predict_lsa(text, summary, lsa_summarizer, tokenizer, summary_size=3):
    parser = PlaintextParser.from_string(text, tokenizer)
    predicted_summary = lsa_summarizer(parser.document, summary_size)
    predicted_summary = " ".join([str(s) for s in predicted_summary])
    return predicted_summary

In [30]:
lsa_summarizer = LsaSummarizer()
tokenizer = Tokenizer("russian")
calc_method_score(test_records, lambda x, y: predict_lsa(x, y, lsa_summarizer, tokenizer))

Count: 1000
Ref: Телеканал «Спас» запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер. Организаторы отметили, что это беспрецедентный подобный проект на телевидении. Участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: Проживи месяц в Ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект. Известный миссионер, председатель Синодального отдела МП по взаимодействию с Вооруженными силами и правоохранительными учреждениями протоиерей Дмитрий Смирнов выразил надежду, что проект даст благие результаты. По его словам, такая зависимость сродни алкогольной или наркотической — электронные устройства лишают человека свободы и приводят к «дегуманизации» личности, уводя в виртуальную реальность.
BLEU:  0.2886703084253367
ROUGE:  {'rouge-1': {'f': 0.1423559861468858, 'p':

(0.2886703084253367,
 {'rouge-1': {'f': 0.1423559861468858,
   'p': 0.11727499491487438,
   'r': 0.18903538180743218},
  'rouge-2': {'f': 0.030348984789405405,
   'p': 0.024847985847283288,
   'r': 0.04082700623403083},
  'rouge-l': {'f': 0.11588432352670633,
   'p': 0.10587937702733088,
   'r': 0.17031287959819613}})

In [31]:
calc_method_score(test_records, lambda x, y: predict_lsa(x, y, lsa_summarizer, tokenizer, summary_size=1))

Count: 1000
Ref: Телеканал «Спас» запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер. Организаторы отметили, что это беспрецедентный подобный проект на телевидении. Участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: Известный миссионер, председатель Синодального отдела МП по взаимодействию с Вооруженными силами и правоохранительными учреждениями протоиерей Дмитрий Смирнов выразил надежду, что проект даст благие результаты.
BLEU:  0.2015078993309207
ROUGE:  {'rouge-1': {'f': 0.1058174013971358, 'p': 0.1364973645436674, 'r': 0.08954748564495947}, 'rouge-2': {'f': 0.01986588518829345, 'p': 0.026424120032844774, 'r': 0.016478189076842086}, 'rouge-l': {'f': 0.08369034967010418, 'p': 0.11937351661944794, 'r': 0.07800626413579102}}


(0.2015078993309207,
 {'rouge-1': {'f': 0.1058174013971358,
   'p': 0.1364973645436674,
   'r': 0.08954748564495947},
  'rouge-2': {'f': 0.01986588518829345,
   'p': 0.026424120032844774,
   'r': 0.016478189076842086},
  'rouge-l': {'f': 0.08369034967010418,
   'p': 0.11937351661944794,
   'r': 0.07800626413579102}})

Здесь уменьшение длины саммари тоже ухудшает метрики, да и сам текст саммари не очень хорошо описывает главную мысль статьи

In [32]:
rs = calc_method_score(test_records, lambda x, y: predict_lsa(x, y, lsa_summarizer, tokenizer))
res['lsa'] = rs[0], rs[1]['rouge-1']['f']
res

Count: 1000
Ref: Телеканал «Спас» запускает реалити-шоу «Остров», участникам которого предстоит месяц жить и работать в Нило-Столобенской пустыни на озере Селигер. Организаторы отметили, что это беспрецедентный подобный проект на телевидении. Участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: Проживи месяц в Ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект. Известный миссионер, председатель Синодального отдела МП по взаимодействию с Вооруженными силами и правоохранительными учреждениями протоиерей Дмитрий Смирнов выразил надежду, что проект даст благие результаты. По его словам, такая зависимость сродни алкогольной или наркотической — электронные устройства лишают человека свободы и приводят к «дегуманизации» личности, уводя в виртуальную реальность.
BLEU:  0.2886703084253367
ROUGE:  {'rouge-1': {'f': 0.1423559861468858, 'p':

{'lead-1': (0.19177311186434495, 0.23804097238957525),
 'textrank': (0.23481140598760156, 0.1709577523245075),
 'lextrank': (0.3585872220880093, 0.1866598392888999),
 'lsa': (0.2886703084253367, 0.1423559861468858)}

### Summa

In [33]:
def calc_summa_score(records, summary_part=0.1, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in tqdm(enumerate(records)):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        predicted_summary = summarize(text, ratio=summary_part, language='russian').replace("\n", " ")
        predictions.append(predicted_summary)

    return calc_scores(references, predictions)

In [34]:
calc_summa_score(test_records, summary_part=0.1)

0it [00:00, ?it/s]

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: проживи месяц в ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект. «у нас же даже многие журналисты не знают и не понимают многого, связанного с религиозными ценностями, а здесь — попытка обратить их внимание на это, может быть, им будет интересно. стоит отметить, что участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека. и это что — жизнь, что ли?
BLEU:  0.2762958192799957
ROUGE:  {'rouge-1': {'f': 0.1770425497877709, 'p': 0.14391909693539454, 'r': 0.247378763312854

(0.2762958192799957,
 {'rouge-1': {'f': 0.1770425497877709,
   'p': 0.14391909693539454,
   'r': 0.24737876331285494},
  'rouge-2': {'f': 0.047316399226292105,
   'p': 0.03775600414501152,
   'r': 0.06889084948510987},
  'rouge-l': {'f': 0.13869575224542852,
   'p': 0.12795603134013936,
   'r': 0.2198206686289155}})

In [35]:
calc_summa_score(test_records, summary_part=0.2)

0it [00:00, ?it/s]

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области. проживи месяц в ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект. в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. известный миссионер, председатель синодального отдела мп по взаимодействию с вооруженными силами и правоохранительными учреждениями протоиерей дмитрий смирн

(0.17989347848877807,
 {'rouge-1': {'f': 0.16610940781957212,
   'p': 0.11122157352562903,
   'r': 0.3504742736856672},
  'rouge-2': {'f': 0.04791529285523528,
   'p': 0.030941039937100664,
   'r': 0.1140776493567048},
  'rouge-l': {'f': 0.10866960085540806,
   'p': 0.10132825147516193,
   'r': 0.3197525014804506}})

С увеличением длины саммари опять видим просадку в bleu и небольшой рост rouge. При этом текст саммари становится более похож по смыслу с исходным

In [36]:
rs = calc_summa_score(test_records, summary_part=0.2)
res['summa'] = rs[0], rs[1]['rouge-1']['f']
res

0it [00:00, ?it/s]

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области. проживи месяц в ниловой пустыни, выполняя послушания, и найди ответы на вопросы, которые давно беспокоят», — так анонсирует телеканал свой проект. в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. известный миссионер, председатель синодального отдела мп по взаимодействию с вооруженными силами и правоохранительными учреждениями протоиерей дмитрий смирн

{'lead-1': (0.19177311186434495, 0.23804097238957525),
 'textrank': (0.23481140598760156, 0.1709577523245075),
 'lextrank': (0.3585872220880093, 0.1866598392888999),
 'lsa': (0.2886703084253367, 0.1423559861468858),
 'summa': (0.17989347848877807, 0.16610940781957212)}

### Oracle summary

In [37]:
def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    
    score = -1.0
    summaries = []
    for _ in range(n_sentences):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [38]:
def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for i, record in tqdm(enumerate(records)):
        if i >= nrows:
            break

        summary = record["summary"]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    return calc_scores(references, predictions)

In [39]:
# calc_oracle_score(test_records)

In [40]:
rs = calc_oracle_score(test_records)
res['oracle_sum'] = rs[0], rs[1]['rouge-1']['f']
res

0it [00:00, ?it/s]

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области. в комментарии также отмечается, что это беспрецедентный подобный проект на телевидении. стоит отметить, что участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
BLEU:  0.531336150784986
ROUGE:  {'rouge-1': {'f': 0.36951810858804146, 'p': 0.4053281117404892, 'r': 0.3661389123393327}, 'rouge-2': {'f': 0.2087846693590

{'lead-1': (0.19177311186434495, 0.23804097238957525),
 'textrank': (0.23481140598760156, 0.1709577523245075),
 'lextrank': (0.3585872220880093, 0.1866598392888999),
 'lsa': (0.2886703084253367, 0.1423559861468858),
 'summa': (0.17989347848877807, 0.16610940781957212),
 'oracle_sum': (0.531336150784986, 0.36951810858804146)}

### RNN

In [41]:
df_train = pd.DataFrame(train_records)
df_test = pd.DataFrame(test_records)
df_test[:3]

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08


In [42]:
df_train['summary_clean'] = df_train['summary'].apply(lambda v: 'BOS ' + v + ' EOS')
df_test['summary_clean'] = df_test['summary'].apply(lambda v: 'BOS ' + v + ' EOS')
df_train['summary_clean'][:5]

0    BOS В 2011 году из России уйдет $80 млрд, счит...
1    BOS Юлия Соловьева, экс-директор холдинга «Про...
2    BOS Суд арестовал на два месяца четверых экс-ч...
3    BOS Мнения предпринимателей по поводу вступлен...
4    BOS На Первом канале завершился третий сезон ш...
Name: summary_clean, dtype: object

In [43]:
MAX_TRAIN_SAMPLE = 5000
MAX_TEST_SAMPLE = 500

df_train = df_train[:MAX_TRAIN_SAMPLE]
df_test = df_test[:MAX_TEST_SAMPLE]

In [44]:
max_len_text = 1000
max_len_sum = 100

In [45]:
tok_text = tf.keras.preprocessing.text.Tokenizer(oov_token='UNK')
tok_text.fit_on_texts(df_train['text'])
x_train_tok = tok_text.texts_to_sequences(df_train['text'])
x_test_tok = tok_text.texts_to_sequences(df_test['text'])

In [46]:
text_vocab_size=len(tok_text.word_index)+1

padded_x_train = pad_sequences(x_train_tok, maxlen=max_len_text, padding='post', truncating='post')
padded_x_test = pad_sequences(x_test_tok, maxlen=max_len_text, padding='post', truncating='post')

In [47]:
tok_sum = tf.keras.preprocessing.text.Tokenizer(oov_token='UNK')
tok_sum.fit_on_texts(df_train['summary_clean'])
x_train_sum = tok_sum.texts_to_sequences(df_train['summary_clean'])
x_test_sum = tok_sum.texts_to_sequences(df_test['summary_clean'])

sum_vocab_size=len(tok_sum.word_index)+1

padded_x_train_sum = pad_sequences(x_train_sum, maxlen=max_len_sum, padding='post', truncating='post')
padded_x_test_sum = pad_sequences(x_test_sum, maxlen=max_len_sum, padding='post', truncating='post')

In [48]:
padded_x_train.shape, padded_x_train_sum.shape

((5000, 1000), (5000, 100))

In [49]:
reverse_text_index=tok_text.index_word
reverse_sum_index=tok_sum.index_word
sum_wordindex=tok_sum.word_index

In [50]:
reverse_sum_index

{1: 'UNK',
 2: 'в',
 3: 'bos',
 4: 'eos',
 5: 'и',
 6: 'на',
 7: 'с',
 8: 'по',
 9: 'не',
 10: 'что',
 11: 'за',
 12: 'из',
 13: '—',
 14: 'а',
 15: 'о',
 16: 'россии',
 17: 'к',
 18: 'от',
 19: 'его',
 20: 'для',
 21: 'сша',
 22: 'но',
 23: 'как',
 24: 'после',
 25: 'может',
 26: 'будет',
 27: 'года',
 28: 'до',
 29: 'он',
 30: 'у',
 31: 'это',
 32: 'также',
 33: 'уже',
 34: 'эксперты',
 35: 'во',
 36: 'при',
 37: 'который',
 38: 'заявил',
 39: 'время',
 40: 'со',
 41: 'этом',
 42: 'лет',
 43: 'году',
 44: 'их',
 45: 'президент',
 46: 'под',
 47: 'ее',
 48: 'ранее',
 49: 'еще',
 50: 'против',
 51: 'президента',
 52: 'матче',
 53: 'более',
 54: 'был',
 55: 'мира',
 56: 'все',
 57: 'однако',
 58: 'украины',
 59: 'словам',
 60: 'только',
 61: 'которые',
 62: 'об',
 63: '1',
 64: 'страны',
 65: 'власти',
 66: 'то',
 67: 'того',
 68: 'суд',
 69: 'считают',
 70: 'владимир',
 71: 'млн',
 72: 'пока',
 73: 'глава',
 74: 'над',
 75: 'компании',
 76: 'чтобы',
 77: 'том',
 78: 'российский',
 79: 

In [51]:
latent_dim = 300
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(max_len_text,))

#embedding layer
enc_emb =  Embedding(text_vocab_size, embedding_dim, trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.0)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim, return_sequences=True,return_state=True,dropout=0.4, recurrent_dropout=0.0)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.0)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) #encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(sum_vocab_size, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.0)
decoder_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(sum_vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1000, 200)    44190200    input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 1000, 300),  601200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [52]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

In [53]:
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
checkpoint_path = "./cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [55]:
# history = model.fit([padded_x_train, padded_x_train_sum[:,:-1]], padded_x_train_sum.reshape(padded_x_train_sum.shape[0],padded_x_train_sum.shape[1], 1)[:,1:],
#                     epochs=50,
#                     validation_data=([padded_x_test, padded_x_test_sum[:,:-1]], padded_x_test_sum.reshape(padded_x_test_sum.shape[0], padded_x_test_sum.shape[1], 1)[:,1:]),
#                     batch_size=16, callbacks=[cp_callback])

Epoch 1/50
313/313 [==============================] - 346s 1s/step - loss: 4.3883 - accuracy: 0.5783 - val_loss: 4.2385 - val_accuracy: 0.5473

Epoch 00001: saving model to .\cp.ckpt
Epoch 2/50
313/313 [==============================] - 340s 1s/step - loss: 3.7124 - accuracy: 0.5924 - val_loss: 4.2193 - val_accuracy: 0.5494

Epoch 00002: saving model to .\cp.ckpt
Epoch 3/50
313/313 [==============================] - 340s 1s/step - loss: 3.6023 - accuracy: 0.5946 - val_loss: 4.1813 - val_accuracy: 0.5525

Epoch 00003: saving model to .\cp.ckpt
Epoch 4/50
313/313 [==============================] - 336s 1s/step - loss: 3.4949 - accuracy: 0.5964 - val_loss: 4.1568 - val_accuracy: 0.5537

Epoch 00004: saving model to .\cp.ckpt
Epoch 5/50
313/313 [==============================] - 335s 1s/step - loss: 3.3758 - accuracy: 0.5998 - val_loss: 4.1327 - val_accuracy: 0.5569

Epoch 00005: saving model to .\cp.ckpt
Epoch 6/50
313/313 [==============================] - 338s 1s/step - loss: 3.2458 - a


Epoch 00045: saving model to .\cp.ckpt
Epoch 46/50
313/313 [==============================] - 333s 1s/step - loss: 0.5986 - accuracy: 0.8850 - val_loss: 4.7453 - val_accuracy: 0.5640

Epoch 00046: saving model to .\cp.ckpt
Epoch 47/50
313/313 [==============================] - 334s 1s/step - loss: 0.5816 - accuracy: 0.8881 - val_loss: 4.7674 - val_accuracy: 0.5633

Epoch 00047: saving model to .\cp.ckpt
Epoch 48/50
313/313 [==============================] - 334s 1s/step - loss: 0.5671 - accuracy: 0.8906 - val_loss: 4.7777 - val_accuracy: 0.5632

Epoch 00048: saving model to .\cp.ckpt
Epoch 49/50
313/313 [==============================] - 333s 1s/step - loss: 0.5531 - accuracy: 0.8929 - val_loss: 4.7917 - val_accuracy: 0.5628

Epoch 00049: saving model to .\cp.ckpt
Epoch 50/50
313/313 [==============================] - 334s 1s/step - loss: 0.5411 - accuracy: 0.8955 - val_loss: 4.8051 - val_accuracy: 0.5634

Epoch 00050: saving model to .\cp.ckpt


In [56]:
model.load_weights(checkpoint_path)

In [57]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text, latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [58]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = sum_wordindex['bos']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_sum_index[sampled_token_index]
        
        if(sampled_token!='eos'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eos'  or len(decoded_sentence.split()) >= (max_len_sum - 1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [59]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=sum_wordindex['bos']) and i!=sum_wordindex['eos']):
            newString=newString+reverse_sum_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_text_index[i]+' '
    return newString

In [60]:
for i in range(0, 3):
    print("Original summary:",seq2summary(padded_x_test_sum[i]))
    print("Predicted summary:",decode_sequence(padded_x_test[i].reshape(1, max_len_text)))

Original summary: в nasa назвали четыре миссии в дальний космос которые в этом UNK могут быть запущены американцами среди них — две миссии по UNK UNK полет к UNK UNK и UNK к UNK UNK UNK 
Predicted summary:  в москве в среду состоялся товарищеский матч «шведских игр» в санкт петербурге между тем самым проблемы с мячом красногорцы большую часть и вид военнослужащим региона и пекин
Original summary: 25 и 26 февраля в кремлевском дворце съездов праздновали UNK — восточный новый год UNK центр российского UNK и один из UNK регионов страны где новый год UNK официально дважды 
Predicted summary:  в москве в среду состоялся товарищеский матч «шведских игр» в санкт петербурге между тем самым проблемы с мячом красногорцы большую часть и вид военнослужащим региона и пекин
Original summary: в белоруссии в день годовщины октябрьской революции выбрали членов верхней палаты парламента сегодняшнее голосование — UNK основной части UNK процесса через десять дней в республике пройдут выборы в палату предс

Предикт выглядит достаточно адекватным, но одинаков для разных текстов

In [61]:
def calc_rnn_score(padded_x, padded_x_sum, max_len_text = 1000):
    references = []
    predictions = []

    for i, rec in tqdm(enumerate(zip(padded_x, padded_x_sum))):
        summary = seq2summary(rec[1])
        references.append(summary)

        text = seq2text(rec[0])
        predicted_summary = decode_sequence(rec[0].reshape(1, max_len_text))
        predictions.append(predicted_summary)

    return calc_scores(references, predictions)

In [62]:
calc_rnn_score(padded_x_test, padded_x_test_sum)

0it [00:00, ?it/s]

Count: 500
Ref: в россии могут ужесточить наказание за оскорбления которые UNK честь или UNK граждан соответствующее поручение генпрокуратуре UNK и UNK суду дал президент рф владимир путин UNK необходимо UNK уголовные дела UNK по факту оскорблений в интернете и предложить варианты как можно усилить ответственность за данное UNK предоставить отчет о UNK работе чиновники должны 1 июля 2020 года 
Hyp:  в москве судят обвиняемых в связи с последствиями наводнения в школах ставрополья станет увольнять из числа окружных условия собственную собственности ее жизнь и это необходимо к переговорам а приштина и вовсе настаивает на подключении к ним она отказалась от налогов до февраля
BLEU:  0.16368421950934656
ROUGE:  {'rouge-1': {'f': 0.08346076997691883, 'p': 0.0952306161997238, 'r': 0.07802339777188758}, 'rouge-2': {'f': 0.0014309253763153786, 'p': 0.001863335637404957, 'r': 0.0012263863245157438}, 'rouge-l': {'f': 0.05909192877598927, 'p': 0.07135466015214312, 'r': 0.05841562168577398}}


(0.16368421950934656,
 {'rouge-1': {'f': 0.08346076997691883,
   'p': 0.0952306161997238,
   'r': 0.07802339777188758},
  'rouge-2': {'f': 0.0014309253763153786,
   'p': 0.001863335637404957,
   'r': 0.0012263863245157438},
  'rouge-l': {'f': 0.05909192877598927,
   'p': 0.07135466015214312,
   'r': 0.05841562168577398}})

In [63]:
rs = calc_rnn_score(padded_x_test, padded_x_test_sum)
res['rnn'] = rs[0], rs[1]['rouge-1']['f']
res

0it [00:00, ?it/s]

Count: 500
Ref: в россии могут ужесточить наказание за оскорбления которые UNK честь или UNK граждан соответствующее поручение генпрокуратуре UNK и UNK суду дал президент рф владимир путин UNK необходимо UNK уголовные дела UNK по факту оскорблений в интернете и предложить варианты как можно усилить ответственность за данное UNK предоставить отчет о UNK работе чиновники должны 1 июля 2020 года 
Hyp:  в москве судят обвиняемых в связи с последствиями наводнения в школах ставрополья станет увольнять из числа окружных условия собственную собственности ее жизнь и это необходимо к переговорам а приштина и вовсе настаивает на подключении к ним она отказалась от налогов до февраля
BLEU:  0.16368421950934656
ROUGE:  {'rouge-1': {'f': 0.08346076997691883, 'p': 0.0952306161997238, 'r': 0.07802339777188758}, 'rouge-2': {'f': 0.0014309253763153786, 'p': 0.001863335637404957, 'r': 0.0012263863245157438}, 'rouge-l': {'f': 0.05909192877598927, 'p': 0.07135466015214312, 'r': 0.05841562168577398}}


{'lead-1': (0.19177311186434495, 0.23804097238957525),
 'textrank': (0.23481140598760156, 0.1709577523245075),
 'lextrank': (0.3585872220880093, 0.1866598392888999),
 'lsa': (0.2886703084253367, 0.1423559861468858),
 'summa': (0.17989347848877807, 0.16610940781957212),
 'oracle_sum': (0.531336150784986, 0.36951810858804146),
 'rnn': (0.16368421950934656, 0.08346076997691883)}

In [64]:
pd.DataFrame.from_dict(res, orient='index', columns = ['BLEU', 'ROUGE-1']).sort_values(by = 'ROUGE-1', ascending = False)

,BLEU,ROUGE-1
oracle_sum,0.531336,0.369518
lead-1,0.191773,0.238041
lextrank,0.358587,0.186660
textrank,0.234811,0.170958
summa,0.179893,0.166109
lsa,0.288670,0.142356
rnn,0.163684,0.083461


RNN на данный момент имеет самые низкие метрики и немного бессвязные саммари. Лидером по метрикам оказался метод oracle summary. Интересно, что в некоторых методах (textrank, LSA) увеличение длины саммари просаживает метрику bleu, не сильно увеличивая rouge-1, но текст саммари по смыслу становится более близким к оригинальному тексту